In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("data.csv")
data.head()
data.describe()
print(data.shape)
data.isnull().sum()
data.isnull().sum().sum()

(569, 33)


569

In [ ]:
# last colum is all null so remove it
data.drop(data.columns[[32]],axis=1,inplace=True)
data.shape

(569, 32)

In [ ]:
data.diagnosis.value_counts().plot(kind="bar", width=0.1, color=["lightgreen", "cornflowerblue"], legend=1, figsize=(8, 5))
plt.xlabel("(0 = Benign) (1 = Malignant)", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.xticks(fontsize=12);
plt.yticks(fontsize=12)
plt.legend(["Benign"], fontsize=12)
plt.show()

In [ ]:
data['diagnosis'].unique()
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(data['diagnosis'])
y = pd.Series(y)
X = data.drop(data.columns[[0,1]],axis=1,inplace=False)
print(type(y))

<class 'pandas.core.series.Series'>


In [ ]:
plt.figure(figsize=(20, 8))
data_new = pd.concat([y,X],axis=1)
data_new.rename(columns = {0:"TARGET"},inplace=True)
data_new.drop('TARGET', axis=1).corrwith(data_new.TARGET).plot(kind='bar', grid=True, title="Correlation of Squared Error Features with TARGET", color="blue")

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)
print("X_train_0.shape, y_train.shape", X_train.shape, y_train.shape)
print("X_test_0.shape, y_test.shape", X_test.shape, y_test.shape)

X_train_0.shape, y_train.shape (455, 30) (455,)
X_test_0.shape, y_test.shape (114, 30) (114,)


**Feature reduction**

There are various popular methods for feature reduction; the feature selection technique demonstrated here comprises two methods applied in sequence:



1.   Univariate feature reduction (remove low correlations with the target).
2.   Feature reduction based on collinearity (for each highly correlated pair, use only the feature that correlates better with the target value).




In [ ]:
def corr_matrix(y,X, isPlot):
  yX = pd.concat([y, X], axis=1)
  yX = yX.rename(columns = {0: 'TARGET'})
  yX_corr = yX.corr(method = 'pearson').round(2)
  #abs(corr_mat.TARGET).nlargest(2)
  yX_abs_corr = np.abs(yX_corr)
  if isPlot:
    #plt.figure(figsize=(10,10))
    #sns.heatmap(data = yX_abs_corr,annot=True)
    #plt.show()
    plt.figure(figsize=(10, 10))
    plt.imshow(yX_abs_corr, cmap='RdYlGn', interpolation='none', aspect='auto')
    plt.colorbar()
    plt.xticks(range(len(yX_abs_corr)), yX_abs_corr.columns, rotation='vertical')
    plt.yticks(range(len(yX_abs_corr)), yX_abs_corr.columns);
    plt.suptitle('Pearson Correlation Heat Map (absolute values)', fontsize=15, fontweight='bold')
    plt.show()
  return yX, yX_corr, yX_abs_corr

yX, yX_corr, yX_abs_corr = corr_matrix(y_train,X_train,isPlot=True)

1 Univariate feature reduction (remove low correlations with the target)

In [ ]:
#print(np.abs(yX_abs_corr.TARGET).nlargest(3))
sort_corr_y = yX_abs_corr['TARGET'].sort_values(ascending=False)
sort_corr_y[sort_corr_y<=0.1]

fractal_dimension_se      0.07
smoothness_se             0.07
fractal_dimension_mean    0.04
texture_se                0.02
symmetry_se               0.02
Name: TARGET, dtype: float64

Find features that have a low correlation with Target Variable and remove them

In [ ]:
xy = pd.concat([y_train,X_train],axis=1)
corr_xy = xy.corr(method='pearson').abs()
corr_xy = corr_xy.rename(columns = {0:"TARGET"},index={0:"TARGET"})
corr_target = corr_xy['TARGET'].abs()
corr_target.sort_values(ascending=False,inplace=True)
# remove those feature that has correlation less than 0.1 with target (y) variable
to_drop_target  = corr_target[corr_target < 0.1]
#print(corr_target)

remain_cols = list(set(X_train.columns)-set(to_drop_target.index))
print(len(remain_cols))

#remove these features from X_train data
print(X_train.shape)
X_train_new = X_train[remain_cols]
print(X_train_new.shape)

25
(455, 30)
(455, 25)


In [ ]:
# now we create corr for X_train_new and Y and remove those columns which are highly correlated
df_xy = pd.concat([y_train, X_train_new],axis=1)
df_xy.rename(columns = {0:"TARGET"},index={0:"TARGET"},inplace=True)
corr_matrix = X_train_new.corr().abs()
print(X_train_new.shape)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop_corr = [column for column in upper.columns if any(upper[column] > 0.9)]
X_train_2 = X_train_new.drop(to_drop_corr, axis=1, inplace=False)
print(X_train_2.shape)

X_test_temp = X_test[remain_cols]
X_test_2 = X_test_temp.drop(to_drop_corr,axis=1,inplace=False)
print(X_test_2.shape)


(455, 25)
(455, 15)
(114, 15)


In [ ]:
corr_x = X.corr(method='pearson').abs()
upper_tri = corr_x.where(np.triu(np.ones(corr_x.shape),k=1).astype(np.bool))
# drop columns with high correlation > 0.95
to_drop = [cl for cl in upper_tri.columns if any(upper_tri[cl]>.95)]
#print(to_drop)
#---------------------------------------
drops = list(to_drop_target.index) + to_drop
X_new_cor = X.drop(drops,axis=1)
print(X_new_cor.shape)
print(X_new_cor.columns)

In [ ]:
from sklearn.feature_selection import SelectKBest,chi2, f_regression,f_classif
fs_chi = SelectKBest(score_func=chi2,k='all')
fit_chi2 = fs_chi.fit(X_train,y_train)
p_values = pd.DataFrame(fit_chi2.pvalues_)
scores = pd.DataFrame(fit_chi2.scores_)
input_variable_names = pd.DataFrame(X_train.columns)
summary_stats = pd.concat([input_variable_names, p_values, scores], axis = 1)
summary_stats.columns = ["input_variable", "p_value", "chi2_score"]
summary_stats.sort_values(by = "p_value", inplace = True)
p_value_threshold = 0.05
score_threshold = 5
selected_variables = summary_stats.loc[(summary_stats["chi2_score"] >= score_threshold) & (summary_stats["p_value"] <= p_value_threshold)]
selected_variables = selected_variables["input_variable"].tolist()
print(len(selected_variables))
X_new_chi = X_train[selected_variables]

17


In [ ]:
fs_classif = SelectKBest(score_func=f_classif,k='all')
fit_classif = fs_classif.fit(X_train,y_train)
cl_pVals = pd.DataFrame(fit_classif.pvalues_)
cl_scores =pd.DataFrame(fit_classif.scores_)
summary_stat_2 = pd.concat([input_variable_names,cl_pVals,cl_scores],axis=1)
summary_stat_2.columns = ["input_variables","p_value","classif_score"]
print(summary_stat_2)
#**************************************
summary_stat_2.sort_values(by="p_value",inplace=True)
selected_variables_2 = summary_stat_2.loc[(summary_stat_2["classif_score"] >= score_threshold) &
                                       (summary_stat_2["p_value"] <= p_value_threshold)]
selected_variables_2 = selected_variables_2["input_variables"].tolist()
print(len(selected_variables_2))
X_new_classif = X_train[selected_variables_2]


In [ ]:
print(len(X_new_chi.columns))
print(X_new_chi.shape)

print(len(X_new_classif.columns))
print(X_new_classif.shape)

print(len(X_new_cor.columns))

# we use the X_new_cor
print(X_new_cor.shape)

Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train_2)
X_test = scaler.fit_transform(X_test_2)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,accuracy_score
trainning_accuracy = []
test_accuracy = []

for k in range(1,15):
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train_2,y_train)
  trainning_accuracy.append(knn.score(X_train_2,y_train))
  test_accuracy.append(knn.score(X_test_2,y_test))
  #*************************************

plt.plot(range(1,15),trainning_accuracy,label = "Tranining Accuracy")
plt.plot(range(1,15),test_accuracy,label = "Testing Accuracy")
plt.ylabel('Accuracy')
plt.xlabel('Number of Neighbers')
plt.legend()
plt.show()

classifier = KNeighborsClassifier(n_neighbors=6)
classifier.fit(X_train_2,y_train)
y_pred = (classifier.predict(X_test_2))
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))



In [ ]:
# logistic regression
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression(random_state=0)
lg.fit(X_train_2,y_train)
print('Accuracy on the training set: {:.3f}'.format(lg.score(X_train_2,y_train)))
print('Accuracy on the training set: {:.3f}'.format(lg.score(X_test_2,y_test)))

y_pred = lg.predict(X_test_2)
from sklearn.metrics import accuracy_score,confusion_matrix,precision_recall_curve
print("---------------")
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))


In [ ]:
# decision tree
from sklearn.tree import DecisionTreeClassifier,export_graphviz
train_accuracy = []
test_accuracy = []
for i in range(1,15):
  dt = DecisionTreeClassifier(max_depth = i,random_state=0)
  dt.fit(X_train_2,y_train)
  train_accuracy.append(dt.score(X_train_2,y_train))
  test_accuracy.append(dt.score(X_test_2,y_test))

plt.plot(range(1,15),train_accuracy,label = "training accuracy")
plt.plot(range(1,15),test_accuracy,label= "testing accuracy ")
plt.ylabel('Accuracy')
plt.xlabel('Max Depth')
plt.legend()




In [ ]:
export_graphviz(dt, out_file='cancerTree.dot', class_names=['malignant','benign'], feature_names=X_train_2.columns, impurity=False, filled=True)
print('Feature importances: {}'.format(dt.feature_importances_))
type(dt.feature_importances_)

#Feature Importance
n_feature = X_train_2.shape[1]
plt.barh(range(n_feature), dt.feature_importances_, align='center')
plt.yticks(np.arange(n_feature), X_train_2.columns)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.show()



In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, criterion='entropy',random_state=0)
rf.fit(X_train_2,y_train)
print('acc for training data: {:.3f}'.format(rf.score(X_train_2,y_train)))
print('acc for test data: {:.3f}'.format(rf.score(X_test_2,y_test)))

y_pred = rf.predict(X_test_2)
cm_rf = confusion_matrix(y_test,y_pred)
print(cm_rf)
print(accuracy_score(y_test, y_pred))

#Feature Importance
n_feature =X_train_2.shape[1]
plt.barh(range(n_feature), rf.feature_importances_, align='center')
plt.yticks(np.arange(n_feature), X_train_2.columns)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.show()


In [ ]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(X_train_2,y_train)
print("training score", svm.score(X_train_2,y_train))
print("testing score", svm.score(X_test_2,y_test))
y_pred = svm.predict(X_test_2)
print("-----------------------------")
print(accuracy_score(y_pred,y_test))
confusion_matrix(y_pred, y_test)

plt.plot(X_train_2.min(axis=0), 'o', label='Min')
plt.plot(X_train_2.max(axis=0), 'v', label='Max')
plt.xlabel('Feature Index')
plt.ylabel('Feature Magnitude in Log Scale')
plt.yscale('log')
plt.legend(loc='upper right')